이전에 작성한 글에서 프로그램은 결과를 내보이는 것이 목적이었습니다.

여기서는 결과가 아닌 과정(상태)을 내보이는 프로그램을 작성하겠습니다.

아래 코드를 보겠습니다.

### 1. Lazy execution

In [1]:
def promise(x):
    yield x


def cons_stream(a, b):
    return (a, promise(b))

In [3]:
s0 = cons_stream(1, cons_stream(2, cons_stream(3, 4)))
s0

(1, <generator object promise at 0x10410e8b8>)

위의 `tuple`이 가진 첫번째 원소인 `1`은 `s0`의 현재 상태입니다. 

그리고 두번째 원소는 `promise` 함수를 통해 값을 [제너레이터](https://wiki.python.org/moin/Generators)로 저장하고 있습니다.

즉, 이 `promise`는 다음 상태에 대한 __약속__일 뿐, 그에대한 계산을 하지 않습니다. 

( '약속'을 함으로써 다음 계산을 미루고 있기 때문에 이런 방법을 `Delayed execution` 또는 `Lazy execution`이라고 합니다. )

이처럼 스트림은 현재 상태를 가지고 다음 상태를 약속하는 차례열의(여기서는 `tuple`로 표현한) 연속입니다.



\* 원래 `SICP`에서는 '제너레이터'가 아닌 프로시저 실행을 지연함으로써 '약속'을 구현합니다.

간단히는, 아래처럼 구현할 수 있습니다.

```scheme
(define (delay proc)
    (lambda () proc))
```

적절한 함수를 만들어 `s0`를 한 번 쭉 뽑아보면,

In [7]:
def display_stream(s):
    if type(s) is tuple:
        print(s[0])
        return display_stream(next(s[1]))
    else:
        print(s)

In [8]:
display_stream(s0)

1
2
3
4


다음으로, 

1. 임의의 정수 a부터 b까지($a < b$) 연속하는 스트림을 만들고,

2. 그 스트림에 특정 조건을 걸어서 필터링시키는 코드를 작성하겠습니다.

In [39]:
def stream_enumerate_interval(low, high):
    if high > low:
        return cons_stream(low, stream_enumerate_interval(low+1, high))
    else:
        return high
    
    
def stream_filter(pred, stream):
    if type(stream) is not tuple:
        return stream
    elif pred(stream[0]):
        return cons_stream(
            stream[0],
            stream_filter(pred, next(stream[1])))
    else:
        return stream_filter(pred, next(stream[1]))

In [40]:
a = stream_enumerate_interval(1, 10)
b = stream_filter(lambda x: x%2 == 0, a)
display_stream(b)

2
4
6
8
10


In [30]:
def t(init_values=()):
    for value in init_values:
        yield value
    yield 0
        
q = t((1, 2, 3))
next(q)
next(q)
next(q)
next(q)
next(q)

StopIteration: 

In [56]:
def mero_f(f):
    (
        lambda is_alreday_run, result:
            lambda:
                result = (f)
                is_already_run = True
        
                if not is_already_run:
                    return result
    )(False, False)

SyntaxError: invalid syntax (<ipython-input-56-41ac3cce4d1f>, line 5)